In [52]:
import numpy as np
from numpy.linalg import norm
from math import pi
import matplotlib.pyplot as plt
from matplotlib import animation
import os

In [ ]:
N = 1024                
phi_max = 150        
L = 2 * phi_max
dphi = L / N
phi = np.array([-phi_max + (j + 0.5) * dphi for j in range(N)])  


V0 = 10      
w = 3        
sep = 20      


c1 = -sep/2.0
c2 = +sep/2.0

In [54]:
k = np.fft.fftfreq(N, d=dphi)   
p = 2 * pi * np.fft.fftshift(k)

In [55]:
def V_of_phi(phi_array, V0=V0, w=w, c1=c1, c2=c2):
    V = np.zeros_like(phi_array)
    
    left1 = c1 - w/2.0
    right1 = c1 + w/2.0
    
    left2 = c2 - w/2.0
    right2 = c2 + w/2.0
    V[(phi_array >= left1) & (phi_array <= right1)] = V0
    V[(phi_array >= left2) & (phi_array <= right2)] = V0
    return V

In [56]:
V = V_of_phi(phi)  

K_diag = 0.5 * p**2

In [57]:

phi0 = -100  
p0 =  30      
sigma = 0.8    

psi = np.exp(-(phi - phi0)**2 / (4 * sigma**2) + 1j * p0 * phi)
psi /= norm(psi)  

In [58]:
T = 30
dt = 0.005            
n_steps = int(T / dt)         
snap_every = 6        
mass = 1.0            

In [59]:
expV_half = np.exp(-1j * V * (dt / 2.0))
expK_full = np.exp(-1j * K_diag * dt)


times = []
expect_phi = []
expect_p = []
norms = []


frames = []

In [60]:
def to_momentum(psi_pos):
    
    arr = np.fft.ifftshift(psi_pos)           
    psi_k = np.fft.fft(arr)                   
    psi_p = np.fft.fftshift(psi_k) / np.sqrt(N)   
    return psi_p

def to_position(psi_p):
    arr = np.fft.ifftshift(psi_p * np.sqrt(N))  
    psi_x = np.fft.ifft(arr)
    psi_pos = np.fft.fftshift(psi_x)            
    return psi_pos

test = np.random.randn(N) + 1j * np.random.randn(N)
test /= norm(test)
if norm(to_position(to_momentum(test)) - test) > 1e-10:
    print("Warning: FFT-based transforms might have a small numerical mismatch.")

In [61]:

psi_current = psi.copy()

for step in range(n_steps):
    t = step * dt

  
    psi_current = expV_half * psi_current

    
    psi_p = to_momentum(psi_current)
    psi_p = expK_full * psi_p
    psi_current = to_position(psi_p)

    
    psi_current = expV_half * psi_current

    
    psi_current /= norm(psi_current)

   
    prob = np.abs(psi_current) ** 2
    times.append(t + dt)
    expect_phi.append(np.sum(prob * phi))
  
    prob_p = np.abs(psi_p) ** 2
    expect_p.append(np.sum(prob_p * p))
    norms.append(np.sum(prob))

  
    if (step % snap_every) == 0:
        frames.append(prob.copy())


In [ ]:
# -------------------- Animation with potential overlay --------------------
fig_anim, ax_anim = plt.subplots(figsize=(8,4))

# scaled potential for visualization
V_plot = V / V.max() * max(frames[0]) * 0.8

# initial frame
line, = ax_anim.plot(phi, frames[0], lw=2, color='C0')
ax_anim.plot(phi, V_plot, 'k--', lw=1.5)

ax_anim.set_xlim(phi[0], phi[-1])
ax_anim.set_ylim(0, max(frames[0].max(), max(fr.max() for fr in frames))*1.2)
ax_anim.set_xlabel(r"$\phi$")
ax_anim.set_ylabel("Probability density")
ax_anim.set_title("Wavepacket scattering on double square barrier (JLP, 0+1D)")
ax_anim.grid(True)

def animate(i):
    line.set_ydata(frames[i])
    return (line,)

anim = animation.FuncAnimation(
    fig_anim, animate,
    frames=len(frames),
    interval=40,
    blit=True
)

gif_path = "double_barrier_with_potential.gif"
anim.save(gif_path, writer=animation.PillowWriter(fps=20))

plt.close(fig_anim)
print("Saved animation with potential overlay to:", gif_path)